# TCGA Run Series 2 - Non-matched
```
pi:ababaian
files: ~/Crown/data2/tcag2_pilot/
start: 2019 05 06
complete : YYYY MM DD
```
## Introduction

I've been working with G. Novakovsky the last few months on a derivative of `Crown` called `Goat` (https://github.com/ababaian/Goat) which is focused on 3-D clustering of rRNA mutations/variations and RP mutations (from COSMIC) in the cryo-EM structure.

We've had quite a bit of interesting hits arise in the last few weeks which need to be validated.

### 28S.G2407

The most notable mutation in rRNA is 28S.G2407C. This variation is recurrent in three patients (out of 649), two of which are Lung Adenocarcinoma.


| hgr1_coord | gene_position | gene | cancer    | ref_allele | alt_allele | VAFcancer   | VAFnorm     | MutOrSNP | DPcancer | DPnorm | patient | patientID    | patientAll | Mut/Patient | Mut/Site | 
|------------|---------------|------|-----------|------------|------------|-------------|-------------|----------|----------|--------|---------|--------------|------------|-------------|----------| 
| 1010354    | 2407          | 28S  | TCGA-LUAD | G          | C,T,A      | 0.167116564 | 0.000470035 | MUT      | 4075     | 8510   | 8       | TCGA-44-2662 | 392        | 7           | 3        | 
| 1010354    | 2407          | 28S  | TCGA-BLCA | G          | C,T,A      | 0.192732455 | 0           | MUT      | 5614     | 3240   | 1       | TCGA-BL-A13J | 1          | 2           | 3        | 
| 1010354    | 2407          | 28S  | TCGA-LUAD | G          | C,T,A      | 0.998996488 | 0           | MUT      | 63776    | 6231   | 15      | TCGA-44-6147 | 399        | 1           | 3        | 

1. The variant allele frequency (VAF) of these variants is high in the cancers which have them (16.7 - 99.8%). Considering there are between 50-800 rDNA copies, a high VAF is consistent with a *sweep* for this allele when the mutation arises. **Note:** This is still confounded by incomplete cancer-cell frequencies in the samples, so the actual allele frequency may be higher.
2. These positions are *de novo* mutations, they are absent from the patient-matched normal control RNA-seq (VAFnorm < 0.001). There is such mutation/variation in any of the remaining normal controls (~645).

![28S.G2407 VAF_Cancer vs. VAF_Normal for 649 patients](../figure2/190506_28S.G2407_VAF.png)


`INCLUDE EXAMPLE OF IGV READ STATS SHOWING BI_DIRECTIONAL MUTATION IN PATIENT`

I will be expanding the TCGA RNAseq analysis to include DNAseq, as well as the remaining patients without matched cancer-normal RNAseq. This will bring up the discovery set to N = ~10,000 cancers. We will use normal DNAseq to identify mutations (vs rDNA SNPs with high allele frequency).

Initially I would like to test `TCGA-44-2662`, `TCGA-BL-A13J`, `TCGA-44-6147` replicate RNA-seq, as well as DNA-seq to validate the presense/absense of G2407C at similiar allele frequency, and validate this mutation.



## Objective

1. Re-initiate AWS S3 based TCGA analysis.
2. Run a pilot alignment on all seq data from `TCGA-44-2662`, `TCGA-BL-A13J`, `TCGA-44-6147` to `hgr1`.


## Matererials and Methods

### RNA-seq and Input Table

#### RNAseq

| File ID                              | File Name                                                 | Data Category       | Data Type     | Project ID | Case ID      | Sample ID        | Sample Type   | Case_Match | MatchedNormal | CanNorm_Pair1 | CanNorm_Pair2 | IncludeSample | 
|--------------------------------------|-----------------------------------------------------------|---------------------|---------------|------------|--------------|------------------|---------------|------------|---------------|---------------|---------------|---------------| 
| 10cdc012-492d-4090-83ab-ff73ac0ee536 | f7f51052-0994-49aa-b55a-f84b21429e80_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-LUAD  | TCGA-44-2662 | TCGA-44-2662-01A | Primary Tumor | 1          | 0             | 0             | 0             | Exclude       | 
| 320b2265-20e7-40a8-a554-f9231f6ea914 | 74be9457-39ce-4029-9076-53b0e57d2067_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-LUAD  | TCGA-44-2662 | TCGA-44-2662-01A | Primary Tumor | 1          | 0             | 0             | 0             | Exclude       | 
| fd1ec26a-64e6-4bc4-b923-182bcf3446a0 | 2775d5a7-9663-4169-83e3-937032a27d78_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-BLCA  | TCGA-BL-A13J | TCGA-BL-A13J-01A | Primary Tumor | 1          | 0             | 0             | 0             | Exclude       | 
| 26a1a274-2b68-4d22-8942-321c718e5bd7 | 786870b0-b32a-4e46-90bf-31792658a685_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-BLCA  | TCGA-BL-A13J | TCGA-BL-A13J-01A | Primary Tumor | 0          | 0             | 0             | 0             | Exclude       | 
| 16dfa0e8-e7cf-4589-bd99-82944d703695 | 6cb90565-9ca5-48ee-a258-40ea601c7380_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-LUAD  | TCGA-44-6147 | TCGA-44-6147-01A | Primary Tumor | 1          | 0             | 0             | 0             | Exclude       | 
| 056c3eb5-5317-4cb2-9f33-0e93b4e54be5 | f493fc9c-abd8-4623-b03b-6300c13a3198_gdc_realn_rehead.bam | Raw Sequencing Data | Aligned Reads | TCGA-LUAD  | TCGA-44-6147 | TCGA-44-6147-01A | Primary Tumor | 0          | 0             | 0             | 0             | Exclude       | 


#### Input Table

| Sample ID        | Project ID | File ID                              | 
|------------------|------------|--------------------------------------| 
| TCGA-44-2662-01A | TCGA-LUAD  | 10cdc012-492d-4090-83ab-ff73ac0ee536 | 
| TCGA-44-2662-01A | TCGA-LUAD  | 320b2265-20e7-40a8-a554-f9231f6ea914 | 
| TCGA-BL-A13J-01A | TCGA-BLCA  | fd1ec26a-64e6-4bc4-b923-182bcf3446a0 | 
| TCGA-BL-A13J-01A | TCGA-BLCA  | 26a1a274-2b68-4d22-8942-321c718e5bd7 | 
| TCGA-44-6147-01A | TCGA-LUAD  | 16dfa0e8-e7cf-4589-bd99-82944d703695 | 
| TCGA-44-6147-01A | TCGA-LUAD  | 056c3eb5-5317-4cb2-9f33-0e93b4e54be5 | 


**Note:** When running all RNA-seq samples, there will be collisions in some replciates as is the case for these three samples. Add a suffix `x` from `a-z` in these cases to distinguish replicate files.

| Sample ID         | Project ID | File ID                              | 
|-------------------|------------|--------------------------------------| 
| TCGA-44-2662-01Aa | TCGA-LUAD  | 10cdc012-492d-4090-83ab-ff73ac0ee536 | 
| TCGA-44-2662-01Ab | TCGA-LUAD  | 320b2265-20e7-40a8-a554-f9231f6ea914 | 
| TCGA-BL-A13J-01Aa | TCGA-BLCA  | fd1ec26a-64e6-4bc4-b923-182bcf3446a0 | 
| TCGA-BL-A13J-01Ab | TCGA-BLCA  | 26a1a274-2b68-4d22-8942-321c718e5bd7 | 
| TCGA-44-6147-01Aa | TCGA-LUAD  | 16dfa0e8-e7cf-4589-bd99-82944d703695 | 
| TCGA-44-6147-01Ab | TCGA-LUAD  | 056c3eb5-5317-4cb2-9f33-0e93b4e54be5 | 



### Scripts and Localization

In [1]:
WORKDIR='/home/artem/Crown/data2/tcga2_pilot'
cd $WORKDIR
ls

# Amazon AWS S3 Home URL
S3URL='s3://crownproject/tcga2'

droneB.sh  hgr1_align_v3.tcga.sh  input_tcga2.txt  pilot_files.xlsx  queenB.sh


In [5]:
INPUT='input_tcga2.txt'

cat $INPUT

TCGA-44-2662-01Aa	TCGA-LUAD	10cdc012-492d-4090-83ab-ff73ac0ee536
TCGA-44-2662-01Ab	TCGA-LUAD	320b2265-20e7-40a8-a554-f9231f6ea914
TCGA-BL-A13J-01Aa	TCGA-BLCA	fd1ec26a-64e6-4bc4-b923-182bcf3446a0
TCGA-BL-A13J-01Ab	TCGA-BLCA	26a1a274-2b68-4d22-8942-321c718e5bd7
TCGA-44-6147-01Aa	TCGA-LUAD	16dfa0e8-e7cf-4589-bd99-82944d703695
TCGA-44-6147-01Ab	TCGA-LUAD	056c3eb5-5317-4cb2-9f33-0e93b4e54be5

#### Scripts

In [3]:
cd $WORKDIR
# Echo 

cat hgr1_align_v3.tcga.sh
echo 
echo
cat queenB.sh
echo 
echo
cat droneB.sh
echo 
echo 

#!/bin/bash
# 1kg_align_v2.tcga.sh
# rDNA alignment pipeline
# 190506 build -- TCGA
# AMI: crown-180813 - ami-0031fd61f932bdef9
# EC2: c4.2xlarge (8cpu / 15 gb)
# EC2: c4.xlarge  (4cpu / 8  gb)
# Storage: 200 Gb
#

# Input Requirements --------------------------

# $1 : Library name and file-output name (unique)
# $2 : Library population/analysis set
# $3 : Library UUID

# Control Panel -------------------------------
# Amazon AWS S3 Home URL
  S3URL='s3://crownproject/tcga2'

# CPU
	THREADS='3'

# Terminate instances upon completion (for debuggin)
  TERMINATE='FALSE'

# Sequencing Data
	LIBRARY=$1 # Library/ File name

# TCGA FILE UUID
  UUID=$3

 # FastQ File-names
    FQ0="$LIBRARY.tmp.sort.0.fq"
    FQ1="$LIBRARY.tmp.sort.1.fq"
    FQ2="$LIBRARY.tmp.sort.2.fq"
    
# Read Group Data
# Extract from downloaded BAM file / input
	RGPO=$2 # Patient Population

	#RGSM= # Sample. Patient Identifer
	#RGID= # Read Group ID. Accession Number
    
	RGLB=$LIBRARY # Library Name. Accession Numb

## Results - TCGA2 Pilot Run



In [6]:
# Local Folder Operations -----------------------------
# LOCAL:
cd $WORKDIR

#NOTE For pilot run, AWS s3 shutdown commented out. Re-upload hgr1 script upon full run

aws s3 cp queenB.sh $S3URL/scripts/
aws s3 cp droneB.sh $S3URL/scripts/
aws s3 cp hgr1_align_v3.tcga.sh $S3URL/scripts/
aws s3 cp $INPUT $S3URL/scripts/
aws s3 cp ../../gdc.token.txt $S3URL/scripts/gdc.token


upload: ./queenB.sh to s3://crownproject/tcga2/scripts/queenB.sh
upload: ./droneB.sh to s3://crownproject/tcga2/scripts/droneB.sh
upload: ./hgr1_align_v3.tcga.sh to s3://crownproject/tcga2/scripts/hgr1_align_v3.tcga.sh
upload: ./input_tcga2.txt to s3://crownproject/tcga2/scripts/input_tcga2.txt
upload: ../../gdc.token.txt to s3://crownproject/tcga2/scripts/gdc.token


In [7]:
# Remote EC2 Instance Operations ----------------------

# Remote:
# Manually open an Amazon Linux 2 AMI
# ami-061392db613a6357b
# t2.micro
#
# ssh login:
# ssh -i "crown.pem" ec2-user@PUBLICDNS
#

# Commands on EC2 machine to set-up AWS
# enter personal login info:

# REMOTE:
#aws configure
  # AWS Key ID
  # AWS Secret Key ID
  # Region: us-west-2
  
# Copy local run files to S3 and download them on EC2

# REMOTE:
# aws s3 cp --recursive s3://crownproject/tcga2/scripts/ ./
#
# mv <KEY>.pem ~/.ssh/
# chmod 400 ~/.ssh/<KEY>.pem

# REMOTE:
# Open logging screen and being launchign EC2 instances
# screen -L
# 
# bash queenB.sh input_tcga2.txt
#
# aws s3 cp screenlog.0 s3://crownproject/tcga2/logs/tcga2_pilot.log

aws s3 cp s3://crownproject/tcga2/logs/tcga2_pilot.log ./
cat tcga2_pilot.log

# Run completed successfully

download: s3://crownproject/tcga2/logs/tcga2_pilot.log to ./tcga2_pilot.log











































































kec2-user@ip-172-31-31-231:~\[ec2-user@ip-172-31-31-231 ~]$ 

## Discussion

### TCGA-44-2662
The variant did not replicate at 2407 in the other sample in this patient.

```
Replicate 01Aa
chr13:1,010,354
<hr>Total count: 729118
A      : 675  (0%,     584+,   91- )
C      : 18901  (3%,     12835+,   6066- )
G      : 708504  (97%,     428912+,   279592- )
T      : 600  (0%,     415+,   185- )
N      : 438  (0%,     392+,   46- )
---------------
DEL: 0
INS: 130
```
```
Replicate 01Ab
chr13:1,010,354
<hr>Total count: 22765
A      : 143  (1%,     139+,   4- )
C      : 160  (1%,     132+,   28- )
G      : 22315  (98%,     18903+,   3412- )
T      : 50  (0%,     22+,   28- )
N      : 97  (0%,     68+,   29- )
---------------
DEL: 0
INS: 1
```

```
Replicate 01B
chr13:1,010,354
<hr>Total count: 4310
A      : 0
C      : 731  (17%,     482+,   249- )
G      : 3570  (83%,     1672+,   1898- )
T      : 9  (0%,     0+,   9- )
N      : 0
---------------
```

```
Replicate 11A (normal)
chr13:1,010,354
<hr>Total count: 9251
A      : 7  (0%,     6+,   1- )
C      : 7  (0%,     4+,   3- )
G      : 9217  (100%,     7425+,   1792- )
T      : 8  (0%,     4+,   4- )
N      : 12  (0%,     7+,   5- )
---------------
```

Exploring reasons or differences between samples on the GDC website I noted that 01Ax is not FFPE slide while 01B is an FFPE slide. This was also true for the other three samples with this variant at 2407. This may perhaps explain some of the variance observed for the alternative allele.


### FFPE Samples
`/home/artem/Desktop/Crown/data2/tcga2_pilot/tcga-ffpe`

I was able to filter for all FFPE samples in TCGA cohorts on the GDC using the filter:
` cases.samples.is_ffpe in [true] and files.data_format in ["BAM"] and files.data_type in ["Aligned Reads"] and files.experimental_strategy in ["RNA-Seq"] `

Which yielded the following list of files:
```
Sample ID
TCGA-BL-A13I-01B
TCGA-A7-A26I-01B
TCGA-AC-A3QQ-01B
TCGA-PL-A8LX-01A
TCGA-A7-A26E-01B
TCGA-A6-2684-01C
TCGA-A7-A26F-01B
TCGA-BK-A0CA-01B
TCGA-BK-A26L-01C
TCGA-AC-A2QH-01B
TCGA-A7-A13G-01B
TCGA-PL-A8LY-01A
TCGA-A7-A0DB-01C
TCGA-44-4112-01B
TCGA-A7-A0DC-01B
TCGA-44-3918-01B
TCGA-AC-A3OD-01B
TCGA-44-2666-01B
TCGA-44-6775-01C
TCGA-44-6147-01B
TCGA-BK-A0CC-01B
TCGA-A6-2674-01B
TCGA-44-6146-01B
TCGA-A6-2677-01B
TCGA-A6-5665-01B
TCGA-PL-A8LZ-01A
TCGA-HC-8258-01B
TCGA-A6-6780-01B
TCGA-B2-5633-01B
TCGA-B2-3923-01B
TCGA-BL-A13J-01B
TCGA-A7-A13D-01B
TCGA-B2-3924-01B
TCGA-A7-A26J-01B
TCGA-A6-3810-01B
TCGA-A6-5659-01B
TCGA-A6-3809-01B
TCGA-44-2656-01B
TCGA-HC-8265-01B
TCGA-44-2662-01B
TCGA-44-2665-01B
TCGA-PL-A8LV-01A
TCGA-A6-2672-01B
TCGA-A6-6781-01B
TCGA-B2-5635-01B
TCGA-PN-A8MA-01A
TCGA-BK-A139-01C
TCGA-A7-A13E-01B
TCGA-44-2668-01B
TCGA-A6-6650-01B
TCGA-A6-5661-01B
TCGA-HC-7740-01B
TCGA-BL-A0C8-01B
TCGA-44-3917-01B
TCGA-44-5645-01B
TCGA-A6-5656-01B
```

Note that all of except 3 (all of which are `TCGA-PL`) of these files are `01B/C` replicates and not `01A` replicates. All further initial variant analyses should focus on comparing `11A` normals to `01A` tumour samples. It is currently unknown how many of these samples are included in the previous analysis cohorts but moving forward all analysis will be on strictly NON-FFPE samples only.


### TCGA Run 2 List

The pipeline is working as it's set-up. Open up the analysis to entire cohorts of patients, avoid repeating samples which have already been performed.

**Master List COAD:**
`cases.project.project_id in ["TCGA-COAD"] and files.data_format in ["BAM"] and files.data_type in ["Aligned Reads"] and files.experimental_strategy in ["RNA-Seq"] and cases.samples.is_ffpe in [false]`

**List of all previously aligned files (1420 files):**
`~/Crown/data2/tcga2_pilot/tcga1_filelist.bamlist`